<a href="https://colab.research.google.com/github/MarcoBonato96/Foundation_ComputerScience_Project/blob/main/Exam_Project_MarcoBonato_def_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Exam

Notes:
- It is mandatory to use GitHub for developing the project.
- The project must be a jupyter notebook.
- There is no restriction on the libraries that can be used, nor on the Python version.
- All questions on the project must be asked in the Discussion forum on the course website.
- At most 3 students can be in each group. You must create the groups by yourself. You can use the Discussion forum to create the groups.
- You do not have to send me the project before the discussion.
- You do not have to prepare any slides for the discussion.
- You can use AI tools, but you have to describe in the notebook how you have used such tools and you have to show that you have fully understood everything that you have in your project.

Public Drive links of the DataSet files:

USvideos.csv.zst: https://drive.google.com/file/d/1P0yEyhX8dIkbFrGgABQOMlmxTlGm6-X5/view?usp=sharing

RUvideos.csv.zst: https://drive.google.com/file/d/1ccU6NER5XBVcv9_a84a_0Db4otGkbAeN/view?usp=drive_link

MXvideos.csv.zst: https://drive.google.com/file/d/1gkqUdibSv4ukSH79eHUbXCT9e8IL1UrY/view?usp=drive_link

KRvideos.csv.zst: https://drive.google.com/file/d/12PReghrtiikWoERoaP2hg8Ex3qW66gDF/view?usp=drive_link

JPvideos.csv.zst: https://drive.google.com/file/d/1h9KJ8uaYs6iRzvZh2lUtKHam67NGmt0x/view?usp=drive_link

INvideos.csv.zst: https://drive.google.com/file/d/14zYUaqCwMeaFH8QBr0r5Z99w5jBpbE2b/view?usp=drive_link

GBvideos.csv.zst: https://drive.google.com/file/d/1OSjxjYOEU6VWerViQGtZpDuEqFAZfANd/view?usp=drive_link

FRvideos.csv.zst: https://drive.google.com/file/d/1cTwcFrPwu1ajdyd4kTapA29Ay1S7uZZk/view?usp=drive_link

DEvideos.csv.zst: https://drive.google.com/file/d/1V2fCdRwq04g3HvSKEMUROfTIvNulvHja/view?usp=drive_link

CAvideos.csv.zst: https://drive.google.com/file/d/1_91yMOw9TTGeuZ-mgzMIh4sVn8w2N7lf/view?usp=drive_link

**AI Usage Statement:** AI (Gemini, Virtual Studio Code Agents) is used in this project EXCLUSIVELY to learn more about Python functions and methods, not to copy and paste scripts, but as a supporting technology.

1. Create a single dataframe with the concatenation of all input csv files, adding a column called **country**

In [4]:
import pandas as pd
import numpy as np
import gdown
from pathlib import Path
import zstandard as zstd # Import zstandard library for decompression
import re

In [5]:
# List of resources (decompressed_name, id, compressed_name), we can enter to each one like RESOURCES[0][0]
RESOURCES = [
    ("USvideos.csv", "1P0yEyhX8dIkbFrGgABQOMlmxTlGm6-X5", "USvideos.csv.zst"),
    ("RUvideos.csv", "1ccU6NER5XBVcv9_a84a_0Db4otGkbAeN", "RUvideos.csv.zst"),
    ("MXvideos.csv", "1gkqUdibSv4ukSH79eHUbXCT9e8IL1UrY", "MXvideos.csv.zst"),
    ("KRvideos.csv", "12PReghrtiikWoERoaP2hg8Ex3qW66gDF", "KRvideos.csv.zst"),
    ("JPvideos.csv", "1h9KJ8uaYs6iRzvZh2lUtKHam67NGmt0x", "JPvideos.csv.zst"),
    ("INvideos.csv", "14zYUaqCwMeaFH8QBr0r5Z99w5jBpbE2b", "INvideos.csv.zst"),
    ("GBvideos.csv", "1OSjxjYOEU6VWerViQGtZpDuEqFAZfANd", "GBvideos.csv.zst"),
    ("FRvideos.csv", "1cTwcFrPwu1ajdyd4kTapA29Ay1S7uZZk", "FRvideos.csv.zst"),
    ("DEvideos.csv", "1V2fCdRwq04g3HvSKEMUROfTIvNulvHja", "DEvideos.csv.zst"),
    ("CAvideos.csv", "1_91yMOw9TTGeuZ-mgzMIh4sVn8w2N7lf", "CAvideos.csv.zst"),
]

# Dictionary of Country
country_codes = {
    'RU': 'Russia', 'MX': 'Mexico', 'US': 'United States',
    'IN': 'India', 'KR': 'South Korea', 'GB': 'United Kingdom',
    'JP': 'Japan', 'FR': 'France', 'CA': 'Canada', 'DE': 'Germany',
}

# File that need a different standard of encoding (using country codes for consistency)
files_with_encoding_issues = {'RU', 'MX', 'KR', 'JP', 'CA'}

all_dataframes = []

# This script below do:
# 1. Extract form the RESOURCES lists the different element (decompressed_filename, id, compressed_name)
# 2. Download the file
# 3. Decompression of the file from .zst format
# 4. Extract the Country corrispondence of each file and search the Country name
# 5. Create the DataFreame of each file
# 6. Create the Column Country for each DataFrame
# 7. Create a unique DataFrame that contain all the csv files

#1
for element in RESOURCES:
    file_id = element[1]
    compressed_filename = element[2]
    decompressed_filename = element[0]
#2
    direct_download_url = f"https://drive.google.com/uc?id={file_id}&export=download"
    gdown.download(url=direct_download_url, output=compressed_filename, quiet=False)

    print(f"Successfully downloaded '{compressed_filename}'. Now decompressing...")
#3
    # Decompress the .zst file
    try:
        with open(compressed_filename, 'rb') as f_in: # "read binary"
            with open(decompressed_filename, 'wb') as f_out: # "write binary"
                dctx = zstd.ZstdDecompressor()
                dctx.copy_stream(f_in, f_out) # read the file from f_in, decompress it and than write the resulted file into f_out
        print(f"Successfully decompressed '{compressed_filename}' to '{decompressed_filename}'.")

    except Exception as e:
        print(f"Error during decompression: {e}")
#4
    # Extract first two letters from decompressed_filename that indicate Country code
    videos_code = Path(decompressed_filename).name[:2].upper()
    # Then extract the exact correspondence of the Country code
    country = country_codes.get(videos_code, 'Unknown')
#5
    # Create the DataFrame and use encoding_errors='replace' for the file that need different encoding
    if videos_code in files_with_encoding_issues:
        df = pd.read_csv(decompressed_filename, encoding='utf-8', encoding_errors='replace', on_bad_lines='skip')
        print(f" Using encoding_errors='replace' for problematic caratcter ")
    else:
        df = pd.read_csv(decompressed_filename, encoding='utf-8', on_bad_lines='skip')
#6
    # Add "Country" coloumn
    df['Country'] = country
#7
    all_dataframes.append(df)

    if all_dataframes:
      df_combined = pd.concat(all_dataframes, ignore_index=True)
      print(f"✓ Combined DataFrame created : {len(df_combined)} rows total")
    else:
      print("\n✗ Error!")

    print("-------------------------------------------------------------------------------------")

Downloading...
From: https://drive.google.com/uc?id=1P0yEyhX8dIkbFrGgABQOMlmxTlGm6-X5&export=download
To: /content/USvideos.csv.zst
100%|██████████| 3.01M/3.01M [00:00<00:00, 45.4MB/s]


Successfully downloaded 'USvideos.csv.zst'. Now decompressing...
Successfully decompressed 'USvideos.csv.zst' to 'USvideos.csv'.
✓ Combined DataFrame created : 40949 rows total
-------------------------------------------------------------------------------------


Downloading...
From: https://drive.google.com/uc?id=1ccU6NER5XBVcv9_a84a_0Db4otGkbAeN&export=download
To: /content/RUvideos.csv.zst
100%|██████████| 9.89M/9.89M [00:00<00:00, 112MB/s]


Successfully downloaded 'RUvideos.csv.zst'. Now decompressing...
Successfully decompressed 'RUvideos.csv.zst' to 'RUvideos.csv'.
 Using encoding_errors='replace' for problematic caratcter 
✓ Combined DataFrame created : 81688 rows total
-------------------------------------------------------------------------------------


Downloading...
From: https://drive.google.com/uc?id=1gkqUdibSv4ukSH79eHUbXCT9e8IL1UrY&export=download
To: /content/MXvideos.csv.zst
100%|██████████| 7.06M/7.06M [00:00<00:00, 45.9MB/s]


...

Successfully downloaded 'FRvideos.csv.zst'. Now decompressing...
Successfully decompressed 'FRvideos.csv.zst' to 'FRvideos.csv'.
✓ Combined DataFrame created : 294221 rows total
-------------------------------------------------------------------------------------


Downloading...
From: https://drive.google.com/uc?id=1V2fCdRwq04g3HvSKEMUROfTIvNulvHja&export=download
To: /content/DEvideos.csv.zst
100%|██████████| 8.94M/8.94M [00:00<00:00, 59.9MB/s]


Successfully downloaded 'DEvideos.csv.zst'. Now decompressing...
Successfully decompressed 'DEvideos.csv.zst' to 'DEvideos.csv'.
✓ Combined DataFrame created : 335061 rows total
-------------------------------------------------------------------------------------


Downloading...
From: https://drive.google.com/uc?id=1_91yMOw9TTGeuZ-mgzMIh4sVn8w2N7lf&export=download
To: /content/CAvideos.csv.zst
100%|██████████| 7.58M/7.58M [00:00<00:00, 39.9MB/s]


Successfully downloaded 'CAvideos.csv.zst'. Now decompressing...
Successfully decompressed 'CAvideos.csv.zst' to 'CAvideos.csv'.
 Using encoding_errors='replace' for problematic caratcter 
✓ Combined DataFrame created : 375942 rows total
-------------------------------------------------------------------------------------


In [ ]:
df_combined['Country'].unique() # Check if there are all the Country

array(['United States', 'Russia', 'Mexico', 'South Korea', 'Japan',
       'India', 'United Kingdom', 'France', 'Germany', 'Canada'],
      dtype=object)

2. Extract all videos that have no tag.


In [7]:
videos_noTag = df_combined[df_combined['tags']=='[none]']
videos_noTag.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,Country
42,NZFhMSgbKKM,17.14.11,Dennis Smith Jr. and LeBron James go back and ...,Ben Rohrbach,17,2017-11-13T15:11:00.000Z,[none],945,7,5,8,https://i.ytimg.com/vi/NZFhMSgbKKM/default.jpg,False,False,False,NaN,United States
97,xfmipNU4Odc,17.14.11,Edna's registered owner thought she was dead f...,Hope For Paws - Official Rescue Channel,15,2017-11-10T18:02:04.000Z,[none],284666,16396,81,949,https://i.ytimg.com/vi/xfmipNU4Odc/default.jpg,False,False,False,Please donate $5 and help us save more lives:\...,United States
133,X7flefV8tec,17.14.11,"President Bill Clinton On Dictators, Democracy...",Team Coco,24,2017-11-09T02:37:49.000Z,[none],366180,4364,4448,1997,https://i.ytimg.com/vi/X7flefV8tec/default.jpg,False,False,False,#ConanNYC Highlight: President Clinton talks a...,United States
136,5x1FAiIq_pQ,17.14.11,Alicia Keys - When You Were Gone,Alicia Keys,10,2017-11-09T15:49:21.000Z,[none],95944,1354,181,117,https://i.ytimg.com/vi/5x1FAiIq_pQ/default.jpg,False,False,False,Find out more in The Vault: http://bit.ly/AK_A...,United States
175,4d07RXYLsJE,17.14.11,Sphaera - demonstrating interaction,Jenny Hanell,28,2017-11-04T20:48:16.000Z,[none],1827,3,0,2,https://i.ytimg.com/vi/4d07RXYLsJE/default.jpg,False,False,False,NaN,United States


3. For each channel, determine the total number of views


In [8]:
# Group by Channel_title and sorting form the bigger one to the smallest one
Channel_views_number = df_combined.groupby('channel_title', as_index=False)['views'].sum().sort_values('views',ascending=False)
Channel_views_number

# the result is the same as
# df_combined.groupby('channel_title', as_index=False).sum()[['channel_title','views']].sort_values('views',ascending=False)

,channel_title,views
4564,ChildishGambinoVEVO,11016766510
15536,Marvel Entertainment,10430605449
17726,NickyJamTV,9479859505
18466,Ozuna,8623329509
28412,ibighit,8205572221
...,...,...
17437,NavylittleMonster,365
25828,Videostendencias,302
17837,No Comment TV,284
22560,Sport Life,163


In [ ]:
# Check if the number of channel_title is correct
len(df_combined['channel_title'].unique())

37824

4. Save all rows with disabled comments and disabled ratings, or that have video_error_or_removed in a new dataframe called excluded, and remove those rows from the original dataframe.

In [9]:
# When there is video_error_or_removed == True --> comments_disabled and ratings_disabled are always == False
excluded = df_combined[(df_combined['comments_disabled']==True) & (df_combined['ratings_disabled']==True) | (df_combined['video_error_or_removed']==True)]
excluded

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,Country
1435,Kn5UgGQukYQ,17.21.11,Breaking Bad's Bryan Cranston on Meeting Charl...,hudsonunionsociety,1,2016-10-14T21:14:51.000Z,"Breaking Bad|""Bryan Cranston""|""malcom in the m...",15058,0,0,0,https://i.ytimg.com/vi/Kn5UgGQukYQ/default.jpg,True,True,False,The Hudson Union www.hudsonunionsociety.com is...,United States
1667,Kn5UgGQukYQ,17.22.11,Breaking Bad's Bryan Cranston on Meeting Charl...,hudsonunionsociety,1,2016-10-14T21:14:51.000Z,"Breaking Bad|""Bryan Cranston""|""malcom in the m...",34207,0,0,0,https://i.ytimg.com/vi/Kn5UgGQukYQ/default.jpg,True,True,False,The Hudson Union www.hudsonunionsociety.com is...,United States
1914,Kn5UgGQukYQ,17.23.11,Breaking Bad's Bryan Cranston on Meeting Charl...,hudsonunionsociety,1,2016-10-14T21:14:51.000Z,"Breaking Bad|""Bryan Cranston""|""malcom in the m...",36137,0,0,0,https://i.ytimg.com/vi/Kn5UgGQukYQ/default.jpg,True,True,False,The Hudson Union www.hudsonunionsociety.com is...,United States
2119,Kn5UgGQukYQ,17.24.11,Breaking Bad's Bryan Cranston on Meeting Charl...,hudsonunionsociety,1,2016-10-14T21:14:51.000Z,"Breaking Bad|""Bryan Cranston""|""malcom in the m...",36579,0,0,0,https://i.ytimg.com/vi/Kn5UgGQukYQ/default.jpg,True,True,False,The Hudson Union www.hudsonunionsociety.com is...,United States
2203,RK_B4Ez4_5Q,17.25.11,Verizon 360 Live: The Macy’s Thanksgiving Day ...,Verizon,24,2017-11-23T19:51:07.000Z,"live stream|""360 video""|""fun videos for kids""|...",2618344,45197,2315,3332,https://i.ytimg.com/vi/RK_B4Ez4_5Q/default.jpg,False,False,True,"This year, we hid special offers throughout th...",United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368315,BEePFpC9qG8,18.07.05,360 Google Doodles/Spotlight Stories: Back to ...,Google Spotlight Stories,1,2018-05-02T13:42:03.000Z,"google spotlight stories|""spotlight stories""|""...",237412,18867,100,1831,https://i.ytimg.com/vi/BEePFpC9qG8/default.jpg,False,False,True,"The Google Doodle, Google Spotlight Stories, G...",Canada
369115,pA9McFNloug,18.11.05,ضرر و زیان ایران پس از خروج آمریکا از برجام,VOA Persian Ofogh,22,2018-05-10T16:22:35.000Z,[none],23836,0,0,0,https://i.ytimg.com/vi/pA9McFNloug/default.jpg,True,True,False,علیرضا نوری زاده، محسن سازگارا.,Canada
369992,QW8whgmyTNU,18.16.05,This Is America: Women's Edit,Nicole Arbour,24,2018-05-12T22:42:24.000Z,"Nicole Arbour|""World's Sexiest Comedian""|""funn...",1415142,0,0,0,https://i.ytimg.com/vi/QW8whgmyTNU/default.jpg,True,True,False,About This Is America: Women’s EditI thought i...,Canada
370255,QW8whgmyTNU,18.17.05,This Is America: Women's Edit,Nicole Arbour,24,2018-05-12T22:42:24.000Z,"Nicole Arbour|""World's Sexiest Comedian""|""funn...",1741312,0,0,0,https://i.ytimg.com/vi/QW8whgmyTNU/default.jpg,True,True,False,About This Is America: Women’s EditI thought i...,Canada


In [10]:
# Remove rows from original DataFrame ()
df_combined = df_combined.drop(index=excluded.index)
df_combined

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,Country
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...,United States
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John...",United States
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,United States
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...,United States
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375937,sGolxsMSGfQ,18.14.06,HOW2: How to Solve a Mystery,Annoying Orange,24,2018-06-13T18:00:07.000Z,"annoying orange|""funny""|""fruit""|""talking""|""ani...",80685,1701,99,1312,https://i.ytimg.com/vi/sGolxsMSGfQ/default.jpg,False,False,False,🚨 NEW MERCH! http://amzn.to/annoyingorange 🚨➤ ...,Canada
375938,8HNuRNi8t70,18.14.06,Eli Lik Lik Episode 13 Partie 01,Elhiwar Ettounsi,24,2018-06-13T19:01:18.000Z,"hkayet tounsia|""elhiwar ettounsi""|""denya okhra...",103339,460,66,51,https://i.ytimg.com/vi/8HNuRNi8t70/default.jpg,False,False,False,► Retrouvez vos programmes préférés : https://...,Canada
375939,GWlKEM3m2EE,18.14.06,KINGDOM HEARTS III – SQUARE ENIX E3 SHOWCASE 2...,Kingdom Hearts,20,2018-06-11T17:30:53.000Z,"Kingdom Hearts|""KH3""|""Kingdom Hearts 3""|""Froze...",773347,25900,224,3881,https://i.ytimg.com/vi/GWlKEM3m2EE/default.jpg,False,False,False,Find out more about Kingdom Hearts 3: https://...,Canada
375940,lbMKLzQ4cNQ,18.14.06,Trump Advisor Grovels To Trudeau,The Young Turks,25,2018-06-13T04:00:05.000Z,"180612__TB02SorryExcuse|""News""|""Politics""|""The...",115225,2115,182,1672,https://i.ytimg.com/vi/lbMKLzQ4cNQ/default.jpg,False,False,False,Peter Navarro isn’t talking so tough now. Ana ...,Canada


5. Add a **like_ratio** column storing the ratio between the number of likes and of dislikes

In [11]:
# Create a new Column named like_ratio
df_combined['like_ratio'] = df_combined['likes'] / df_combined['dislikes']
df_combined.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,Country,like_ratio
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...,United States,19.395482
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John...",United States,15.812724
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,United States,27.352126
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...,United States,15.273273
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,United States,66.483157


6. Cluster the publish time into 10-minute intervals (e.g. from 02:20 to 02:30)

In [12]:
# I need to change the type of "publish_time" from object to datetime
df_combined['publish_time'] = pd.to_datetime(df_combined['publish_time'])
df_combined.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,Country,like_ratio
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13 17:13:01+00:00,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...,United States,19.395482
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13 07:30:00+00:00,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John...",United States,15.812724
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12 19:05:24+00:00,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,United States,27.352126
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13 11:00:04+00:00,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...,United States,15.273273
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12 18:01:41+00:00,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,United States,66.483157


In [13]:
# Create a new column that include publish time into 10-min interval
df_combined['publish_time_interval'] = df_combined['publish_time'].dt.floor('10min')

# Display the new column along with original publish_time to illustrate the clustering
print(df_combined[['publish_time', 'publish_time_interval']])

                    publish_time     publish_time_interval
0      2017-11-13 17:13:01+00:00 2017-11-13 17:10:00+00:00
1      2017-11-13 07:30:00+00:00 2017-11-13 07:30:00+00:00
2      2017-11-12 19:05:24+00:00 2017-11-12 19:00:00+00:00
3      2017-11-13 11:00:04+00:00 2017-11-13 11:00:00+00:00
4      2017-11-12 18:01:41+00:00 2017-11-12 18:00:00+00:00
...                          ...                       ...
375937 2018-06-13 18:00:07+00:00 2018-06-13 18:00:00+00:00
375938 2018-06-13 19:01:18+00:00 2018-06-13 19:00:00+00:00
375939 2018-06-11 17:30:53+00:00 2018-06-11 17:30:00+00:00
375940 2018-06-13 04:00:05+00:00 2018-06-13 04:00:00+00:00
375941 2018-06-13 16:00:03+00:00 2018-06-13 16:00:00+00:00

[373322 rows x 2 columns]


7. For each interval, determine the number of videos, average number of likes and of dislikes.

In [14]:
interval_stats = df_combined.groupby('publish_time_interval').agg( # aggregate
    num_videos=('video_id', 'count'),
    avg_likes=('likes', 'mean'),
    avg_dislikes=('dislikes', 'mean')
).reset_index()

print(interval_stats)

          publish_time_interval  num_videos    avg_likes  avg_dislikes
0     2006-07-23 08:20:00+00:00           1   459.000000      152.0000
1     2007-03-05 16:20:00+00:00           9   336.666667        2.0000
2     2007-06-25 06:50:00+00:00          12   579.833333       11.5000
3     2007-12-03 20:50:00+00:00          16   187.937500       15.6875
4     2008-01-07 21:20:00+00:00          10    99.900000        2.0000
...                         ...         ...          ...           ...
30392 2018-06-14 02:30:00+00:00           1   853.000000       77.0000
30393 2018-06-14 03:00:00+00:00           2  2304.500000       31.5000
30394 2018-06-14 03:20:00+00:00           1  1414.000000       28.0000
30395 2018-06-14 03:30:00+00:00           1  8481.000000      252.0000
30396 2018-06-14 03:40:00+00:00           1   374.000000       24.0000

[30397 rows x 4 columns]


8. For each tag, determine the number of videos.
   Notice that **tags** contains a string with several tags.

In [15]:
# Assuming df has column 'tags' with str like: '"tag1"|"tag2"|"tag3"'
df_exploded = df_combined['tags'].str.split('|').explode()  # divide every string that have "|" separator and explode it in a new row
df_exploded = df_exploded.str.strip().str.strip('"')  #
df_exploded = df_exploded[df_exploded != '']  # delete every empty string

# extract the number of videos in which the tag appears
tag_counts = df_exploded.groupby(df_exploded).size()
print(tag_counts)


tags
! Banii i-au luat mintile                          2
!!                                                 5
!! *me bloquea*                                    1
!!!                                                1
!!!!                                               2
                                                  ..
😥                                                  2
😭 #résistancelamaindesqueezie ❤️                   1
😭قفز عسكري الوحيد من الطائرة العسكرية الجزائرية    1
🚀 smartpixel.com.ua                                1
🤝 ZARBEX zahlt Schutzgeld!!!                       1
Name: tags, Length: 848606, dtype: int64


9. Find the tags with the largest number of videos

In [16]:
# I choose to extract the first 11 tags
tag_counts.sort_values(ascending=False).head(11)

,tags
tags,
[none],37209
funny,15039
comedy,12351
2018,11383
news,6363
music,5909
2017,5688
video,5612
humor,5057


10. For each (**tag, country**) pair, compute average ratio likes/dislikes

In [17]:
# Create a temporary DataFrame that holds the exploded tags and their original indices
tags_with_metadata = pd.DataFrame({
    'tag': df_exploded,
    'Country': df_combined.loc[df_exploded.index, 'Country'],
    'like_ratio': df_combined.loc[df_exploded.index, 'like_ratio']
})
tags_with_metadata

,tag,Country,like_ratio
0,SHANtell martin,United States,19.395482
1,last week tonight trump presidency,United States,15.812724
1,last week tonight donald trump,United States,15.812724
1,john oliver trump,United States,15.812724
1,donald trump,United States,15.812724
...,...,...,...
375941,男友,Canada,4.838710
375941,女友,Canada,4.838710
375941,相處,Canada,4.838710
375941,恐怖情人,Canada,4.838710


In [18]:
# Group by 'tag' and 'Country' and calculate the mean of 'like_ratio'
average_like_ratio_per_tag_country = tags_with_metadata.groupby(['tag', 'Country'])['like_ratio'].mean().reset_index()

display(average_like_ratio_per_tag_country)

,tag,Country,like_ratio
0,! Banii i-au luat mintile,Germany,6.222739
1,!!,France,12.452875
2,!!,Mexico,38.081559
3,!! *me bloquea*,Mexico,65.260870
4,!!!,Mexico,12.952381
...,...,...,...
1097893,😥,Germany,26.384615
1097894,😭 #résistancelamaindesqueezie ❤️,France,10.219048
1097895,😭قفز عسكري الوحيد من الطائرة العسكرية الجزائرية,France,6.223873
1097896,🚀 smartpixel.com.ua,Russia,0.714286


11. For each (**trending_date, country**) pair, the video with the largest number of views

In [19]:
# Create a temporary DataFrame that contains the informations needed
trending_date_df = pd.DataFrame({
    'video_id': df_combined['video_id'],
    'title': df_combined['title'],
    'trending_date': df_combined['trending_date'],
    'Country': df_combined['Country'],
    'views': df_combined['views']
})

trending_date_df.head()

,video_id,title,trending_date,Country,views
0,2kyS6SvSYSE,WE WANT TO TALK ABOUT OUR MARRIAGE,17.14.11,United States,748374
1,1ZAPwfrtAFY,The Trump Presidency: Last Week Tonight with J...,17.14.11,United States,2418783
2,5qpjK5DgCt4,"Racist Superman | Rudy Mancuso, King Bach & Le...",17.14.11,United States,3191434
3,puqaWrEC7tY,Nickelback Lyrics: Real or Fake?,17.14.11,United States,343168
4,d380meD0W0M,I Dare You: GOING BALD!?,17.14.11,United States,2095731


In [20]:
# Group by 'trending_date' and 'Country', then find the index of the row with the maximum 'views' within each group.
idx = trending_date_df.groupby(['trending_date', 'Country'])['views'].idxmax()

# Use the indices to select the corresponding rows from the original DataFrame
video_with_max_views_per_group = trending_date_df.loc[idx]

display(video_with_max_views_per_group)

,video_id,title,trending_date,Country,views
338459,6ZfuNTqbHE8,Marvel Studios' Avengers: Infinity War Officia...,17.01.12,Canada,56367282
257028,3VbHg5fqBYw,Avengers: Infinity War Trailer Tease,17.01.12,France,7281189
297620,6ZfuNTqbHE8,Marvel Studios' Avengers: Infinity War Officia...,17.01.12,Germany,56367282
180485,6ZfuNTqbHE8,Marvel Studios' Avengers: Infinity War Officia...,17.01.12,India,56367282
85084,6ZfuNTqbHE8,Marvel Studios' Avengers: Infinity War Officia...,17.01.12,Mexico,56370607
...,...,...,...,...,...
119368,xTlNMmZKwpA,"Cardi B, Bad Bunny & J Balvin - I Like It [Off...",18.31.05,Mexico,20723565
78753,1nX0kF2UwDc,[BadComedian] - Движение Вверх (Плагиат или ве...,18.31.05,Russia,3125598
154101,ntWSnDV0MYs,[MV] AOA _ Bingle Bangle(빙글뱅글),18.31.05,South Korea,4150448
251102,VYOjWnS4cMY,Childish Gambino - This Is America (Official V...,18.31.05,United Kingdom,217750076


12. Divide **trending_date** into three columns: **year, month, day**

In [21]:
# trending_date is an object --> We need to tranform it into datetime to use dt.year/mont/day method
# Assuming the format is 'YY.DD.MM' (Year.Day.Month), we need to specify it
df_combined['trending_date'] = pd.to_datetime(df_combined['trending_date'], format='%y.%d.%m')
df_combined.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,Country,like_ratio,publish_time_interval
0,2kyS6SvSYSE,2017-11-14,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13 17:13:01+00:00,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...,United States,19.395482,2017-11-13 17:10:00+00:00
1,1ZAPwfrtAFY,2017-11-14,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13 07:30:00+00:00,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John...",United States,15.812724,2017-11-13 07:30:00+00:00
2,5qpjK5DgCt4,2017-11-14,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12 19:05:24+00:00,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,United States,27.352126,2017-11-12 19:00:00+00:00
3,puqaWrEC7tY,2017-11-14,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13 11:00:04+00:00,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...,United States,15.273273,2017-11-13 11:00:00+00:00
4,d380meD0W0M,2017-11-14,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12 18:01:41+00:00,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,United States,66.483157,2017-11-12 18:00:00+00:00


In [22]:
# Create the new Column year, month, day
df_combined['year'] = df_combined['trending_date'].dt.year
df_combined['month'] = df_combined['trending_date'].dt.month
df_combined['day'] = df_combined['trending_date'].dt.day
df_combined.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,...,comments_disabled,ratings_disabled,video_error_or_removed,description,Country,like_ratio,publish_time_interval,year,month,day
0,2kyS6SvSYSE,2017-11-14,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13 17:13:01+00:00,SHANtell martin,748374,57527,2966,...,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...,United States,19.395482,2017-11-13 17:10:00+00:00,2017,11,14
1,1ZAPwfrtAFY,2017-11-14,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13 07:30:00+00:00,"last week tonight trump presidency|""last week ...",2418783,97185,6146,...,False,False,False,"One year after the presidential election, John...",United States,15.812724,2017-11-13 07:30:00+00:00,2017,11,14
2,5qpjK5DgCt4,2017-11-14,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12 19:05:24+00:00,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,...,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,United States,27.352126,2017-11-12 19:00:00+00:00,2017,11,14
3,puqaWrEC7tY,2017-11-14,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13 11:00:04+00:00,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,...,False,False,False,Today we find out if Link is a Nickelback amat...,United States,15.273273,2017-11-13 11:00:00+00:00,2017,11,14
4,d380meD0W0M,2017-11-14,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12 18:01:41+00:00,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,...,False,False,False,I know it's been a while since we did this sho...,United States,66.483157,2017-11-12 18:00:00+00:00,2017,11,14


13. For each (**month, country**) pair, the video with the largest number of views

In [23]:
# Create a temporary DataFrame that contains the informations needed
montly_views_df = pd.DataFrame({
    'video_id': df_combined['video_id'],
    'title': df_combined['title'],
    'month': df_combined['month'],
    'Country': df_combined['Country'],
    'views': df_combined['views']
})

montly_views_df.head()

,video_id,title,month,Country,views
0,2kyS6SvSYSE,WE WANT TO TALK ABOUT OUR MARRIAGE,11,United States,748374
1,1ZAPwfrtAFY,The Trump Presidency: Last Week Tonight with J...,11,United States,2418783
2,5qpjK5DgCt4,"Racist Superman | Rudy Mancuso, King Bach & Le...",11,United States,3191434
3,puqaWrEC7tY,Nickelback Lyrics: Real or Fake?,11,United States,343168
4,d380meD0W0M,I Dare You: GOING BALD!?,11,United States,2095731


In [24]:
# Group by 'month' and 'Country', then find the index of the row with the maximum 'views' within each group
idx_monthViews = montly_views_df.groupby(['month', 'Country'])['views'].idxmax()

# Use the indices to select the corresponding rows from the original DataFrame
video_with_max_views_per_month = montly_views_df.loc[idx_monthViews]

display(video_with_max_views_per_month)

,video_id,title,month,Country,views
346275,LsoLEjrDogU,Bruno Mars - Finesse (Remix) [Feat. Cardi B] [...,1,Canada,43067983
264629,LsoLEjrDogU,Bruno Mars - Finesse (Remix) [Feat. Cardi B] [...,1,France,37728802
305270,LsoLEjrDogU,Bruno Mars - Finesse (Remix) [Feat. Cardi B] [...,1,Germany,37728802
189273,dfnCAmr569k,"Taylor Swift - End Game ft. Ed Sheeran, Future",1,India,42019590
92654,LsoLEjrDogU,Bruno Mars - Finesse (Remix) [Feat. Cardi B] [...,1,Mexico,31680160
...,...,...,...,...,...
86949,FlsCjmMhFmw,YouTube Rewind: The Shape of 2017 | #YouTubeRe...,12,Mexico,100912384
45782,FlsCjmMhFmw,YouTube Rewind: The Shape of 2017 | #YouTubeRe...,12,Russia,52611730
126995,FlsCjmMhFmw,YouTube Rewind: The Shape of 2017 | #YouTubeRe...,12,South Korea,113876217
222770,FlsCjmMhFmw,YouTube Rewind: The Shape of 2017 | #YouTubeRe...,12,United Kingdom,169884583


14. Read all json files with the video categories

RU_category_id.json : https://drive.google.com/file/d/1woFFZPsUHeRwspNLjlaH2Yui9o74ZBfX/view?usp=sharing

US_category_id.json : https://drive.google.com/file/d/1sW1Vwb03IhgJ0qnVezw9KrdSukQ5YgEN/view?usp=sharing

MX_category_id.json : https://drive.google.com/file/d/1pBfSg41EAsmUOFN0lYrAtW0kSxdqcUSP/view?usp=sharing

KR_category_id.json : https://drive.google.com/file/d/1lLxd6a16gT4bt__pDnRKdPYEPcO5Hm79/view?usp=sharing

JP_category_id.json : https://drive.google.com/file/d/1hxFFUrhyfIyzQ0NjGrkF4-ln8lvUipXb/view?usp=sharing

FR_category_id.json : https://drive.google.com/file/d/1bxae8CLRGgDRQiLOZPJ0D8i-GcYwEXb4/view?usp=sharing

IN_category_id.json : https://drive.google.com/file/d/1SLkms4aXtF0P-hbBZW6U40IwHx95NgIm/view?usp=sharing

CA_category_id.json : https://drive.google.com/file/d/1Kzr3zBEFacH9ekrEZt73o_hVzXV50Rax/view?usp=sharing

GB_category_id.json : https://drive.google.com/file/d/1FWPdDb7CeGhcS7ip8jH7Dulh51qcyOmR/view?usp=sharing

DE_category_id.json : https://drive.google.com/file/d/1CiLOyhixSxm8nj7cq3xa7P7rcnbsaFtj/view?usp=sharing


In [37]:
import json

JSON_RESOURCES = [
    ("RU_category_id.json", "1woFFZPsUHeRwspNLjlaH2Yui9o74ZBfX", "RU_category_id"),
    ("US_category_id.json", "1sW1Vwb03IhgJ0qnVezw9KrdSukQ5YgEN", "US_category_id"),
    ("MX_category_id.json", "1pBfSg41EAsmUOFN0lYrAtW0kSxdqcUSP", "MX_category_id"),
    ("KR_category_id.json", "1lLxd6a16gT4bt__pDnRKdPYEPcO5Hm79", "KR_category_id"),
    ("JP_category_id.json", "1hxFFUrhyfIyzQ0NjGrkF4-ln8lvUipXb", "JP_category_id"),
    ("FR_category_id.json", "1bxae8CLRGgDRQiLOZPJ0D8i-GcYwEXb4", "FR_category_id"),
    ("IN_category_id.json", "1SLkms4aXtF0P-hbBZW6U40IwHx95NgIm", "IN_category_id"),
    ("CA_category_id.json", "1Kzr3zBEFacH9ekrEZt73o_hVzXV50Rax", "CA_category_id"),
    ("GB_category_id.json", "1FWPdDb7CeGhcS7ip8jH7Dulh51qcyOmR", "GB_category_id"),
    ("DE_category_id.json", "1CiLOyhixSxm8nj7cq3xa7P7rcnbsaFtj", "DE_category_id"),
]

all_json_df = []

for element in JSON_RESOURCES:
    file_id = element[1]
    final_filename = element[2]
    stating_filename = element[0]
#2
    direct_download_url = f"https://drive.google.com/uc?id={file_id}&export=download"
    gdown.download(url=direct_download_url, output=final_filename, quiet=False)

    print(f"Successfully downloaded '{final_filename}'.")

    with open(final_filename) as f:
      json_df = json.load(f)
    all_json_df.append(json_df)
    print('--------------------------------------------')

category_data_by_country_code = {}
for i, element in enumerate(JSON_RESOURCES):
    country_code = country_codes.get(element[0][:2], 'Unknown') # Extract country code from filename, es. 'RU'
    category_data_by_country_code[country_code] = all_json_df[i] # Assign it


Downloading...
From: https://drive.google.com/uc?id=1woFFZPsUHeRwspNLjlaH2Yui9o74ZBfX&export=download
To: /content/RU_category_id
100%|██████████| 8.22k/8.22k [00:00<00:00, 12.7MB/s]


Successfully downloaded 'RU_category_id'.
--------------------------------------------


Downloading...
From: https://drive.google.com/uc?id=1sW1Vwb03IhgJ0qnVezw9KrdSukQ5YgEN&export=download
To: /content/US_category_id
100%|██████████| 8.50k/8.50k [00:00<00:00, 17.2MB/s]


Successfully downloaded 'US_category_id'.
--------------------------------------------


Downloading...
From: https://drive.google.com/uc?id=1pBfSg41EAsmUOFN0lYrAtW0kSxdqcUSP&export=download
To: /content/MX_category_id
100%|██████████| 8.22k/8.22k [00:00<00:00, 14.8MB/s]


...

Successfully downloaded 'CA_category_id'.
--------------------------------------------


Downloading...
From: https://drive.google.com/uc?id=1FWPdDb7CeGhcS7ip8jH7Dulh51qcyOmR&export=download
To: /content/GB_category_id
100%|██████████| 8.22k/8.22k [00:00<00:00, 7.50MB/s]


Successfully downloaded 'GB_category_id'.
--------------------------------------------


Downloading...
From: https://drive.google.com/uc?id=1CiLOyhixSxm8nj7cq3xa7P7rcnbsaFtj&export=download
To: /content/DE_category_id
100%|██████████| 7.91k/7.91k [00:00<00:00, 16.9MB/s]

Successfully downloaded 'DE_category_id'.
--------------------------------------------


15. For each country, determine how many videos have a category that is not assignable.

In [38]:
rows = []

# Extract from category_data_by_country_code the country code
for country_code in category_data_by_country_code:
  # Create a DataFrame with the information of each Country with Category_id and the value of Assignable
  for item in category_data_by_country_code[country_code]['items']:
    rows.append({
            'Country': country_code,
            'category_id': item['id'],
            'assignable': item['snippet'].get('assignable')
        })

df_categories = pd.DataFrame(rows)
print(df_categories.head())


  Country category_id  assignable
0      DE           1        True
1      DE           2        True
2      DE          10        True
3      DE          15        True
4      DE          17        True


In [39]:
# Before using merge method we need to see if the Column has the same type
print(df_combined[['Country','category_id']].dtypes)
print(df_categories[['Country','category_id']].dtypes)

Country        object
category_id     Int64
dtype: object
Country        object
category_id    object
dtype: object


In [40]:
# Convert category_id in num (coerce transform non-numeric in NaN)
df_combined['category_id'] = pd.to_numeric(df_combined['category_id'], errors='coerce').astype('Int64')
df_categories['category_id'] = pd.to_numeric(df_categories['category_id'], errors='coerce').astype('Int64')


In [41]:
# check if it works well
print(df_combined[['Country','category_id']].dtypes)
print(df_categories[['Country','category_id']].dtypes)

Country        object
category_id     Int64
dtype: object
Country        object
category_id     Int64
dtype: object


In [42]:
# Now merge the two DataFrame with the new Column Assignable
merged_df = pd.merge(df_combined, df_categories, on=['Country','category_id'], how='left', indicator=True)
print(merged_df['_merge'].value_counts())

_merge
left_only     373322
right_only         0
both               0
Name: count, dtype: int64


In [43]:
# for each country extract how many videos have a category that is not assignable
filtered_df = merged_df[merged_df['assignable'] == False]
display(filtered_df.groupby('Country').size())


,0
Country,
